# Frac Production Analysis

In [1]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Production Reports
Note: This is the Colorado well production updated monthly until the end of 2017, broken out by individual well and by production month. 
Data Source: [COGCC Production Summary Data](https://cogcc.state.co.us/documents/data/downloads/production/co%202017%20Annual%20Production%20Summary-xp.zip)

In [2]:
df = pd.read_csv('2017_prod_reports.csv')
print(df.shape)
df.tail()

(857203, 33)


,report_month,report_year,ST,api_county_code,api_seq_num,sidetrack_num,formation_code,well_status,prod_days,water_disp_code,...,gas_prod,btu_sales,gas_press_tbg,gas_press_csg,operator_num,name,facility_name,facility_num,accepted_date,revised
857198,12,2016,5,125,12123,0,NBRR,PR,31.0,C,...,2225.0,1000.0,NaN,NaN,10489,AUGUSTUS ENERGY RESOURCES LLC,Gardner Trust,44-18 2N46W,2017-02-07 14:43:50.530000000,NaN
857199,12,2016,5,125,12124,0,NBRR,PR,31.0,P,...,2364.0,992.0,NaN,NaN,66190,OMIMEX PETROLEUM INC,Fiddler Peak Ranch,4-3-5-45,2017-01-12 16:10:23.057000000,NaN
857200,12,2016,5,125,12125,0,NBRR,PR,31.0,C,...,633.0,1000.0,NaN,NaN,10489,AUGUSTUS ENERGY RESOURCES LLC,Chapman,13-19 1S44W,2017-02-07 14:43:50.530000000,NaN
857201,12,2016,5,125,12126,0,NBRR,PR,31.0,C,...,1202.0,996.0,NaN,NaN,10489,AUGUSTUS ENERGY RESOURCES LLC,Haven Hill,14-15 4N47W,2017-02-07 14:43:50.530000000,NaN
857202,12,2017,5,43,6226,1,NBRR,TA,0.0,NaN,...,NaN,NaN,NaN,NaN,10412,AUSCO PETROLEUM INC,Hudson,1,2017-12-27 14:58:21.907000000,NaN


Let's check for what columns I am dealing with here, along with how many non-null values I have to work with.  

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857203 entries, 0 to 857202
Data columns (total 33 columns):
report_month       857203 non-null int64
report_year        857203 non-null int64
ST                 857203 non-null int64
api_county_code    857203 non-null int64
api_seq_num        857203 non-null int64
sidetrack_num      857203 non-null int64
formation_code     857203 non-null object
well_status        857203 non-null object
prod_days          825707 non-null float64
water_disp_code    454823 non-null object
water_vol          443316 non-null float64
water_press_tbg    240511 non-null float64
water_press_csg    235473 non-null float64
bom_invent         489472 non-null float64
oil_vol            378131 non-null float64
oil_sales          301069 non-null float64
adjustment         50065 non-null float64
eom_invent         487351 non-null float64
gravity_sales      301097 non-null float64
gas_sales          593103 non-null float64
flared             42414 non-null float64
gas

Most of this data is complete for the well information, but a lot of the specific values are missing, leaving approximiately 240K months of production to be analyzed. Let's see if the other sources are any better.

## Frac and Well Data
This data was provided by Jeffrey A. Beunier of Front Range Energy Partners, via the Drilling Info site.  I still haven't been provided any more details about this data, so that's all I know for now.
### Data Indices
This first set of data contains all of the headers for each of the provided datasets.

In [4]:
indices = pd.read_excel('1a_Frac_Capstone/DATA_EXPORT_INDEX.xlsx', usecols=(5, 6, 7, 8, 9))
print(indices.shape)
indices

(105, 5)


,WELL HEADER DATA,PRODUCTION HEADER DATA,PRODUCTION TIME SERIES DATA,WELL TEST DATA,FORMATION TOP DATA
0,API10,NaN,NaN,NaN,NaN
1,API12,API/UWI,Entity ID,API,API
2,API14,Operator Alias,API/UWI,Test Date,Formation
3,Well Name,Well/Lease Name,API/UWI List,Test Formation,Formation Top MD
4,Well Number,Well Number,Monthly Production Date,Test Type,Formation Top TVD
5,Lease Name,Entity Type,Monthly Oil,Liquid Volume,Formation Top Unknown
6,Operator Alias,County/Parish,Monthly Gas,Gas Volume,Formation Bottom Unknown
7,Reported Operator,Reservoir,Monthly Water,Water Volume,Field
8,Field,Production Type,Well Count,Hours Tested,State Province
9,County/Parish,Producing Status,Days,NaN,Basin Name


The Wells data has the majority of columns of interest, and the Production data looks like a much more detailed version of the production data from the COGCC database, which could be interesting, but it was suggested that I look at the first 6 months of production, so I don't need this additional detail.  Finally, the production time series data could be interesting to do modeling using time series to predict future production after that time. The Formations and Test data sets are not very important in this analysis as each well in the Wells data already includes the target formation, and tests do not provide much quality data for our analysis. Let's first look at the Wells dataset.
### Wells Data

In [5]:
# Read in Wells Data for Horizontal wells
wells = pd.read_csv('1a_Frac_Capstone/DJHZ6-17-18Wells.CSV')
wells.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,11,35,36,37,38,39,44,81,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,API10,API12,API14,Well Name,Well Number,Lease Name,Operator Alias,Reported Operator,Field,County/Parish,...,Quarter Quarter,District,Abstract,Block,Survey,Surface Hole Latitude (NAD27),Surface Hole Longitude (NAD27),Bottom Hole Latitude (NAD27),Bottom Hole Longitude (NAD27),UWI
0,"=""0500506864""","=""050050686400""","=""05005068640000""",KISSLER DAN,2-10,KISSLER DAN,S&D,S & D LLC,CHALICE,ARAPAHOE (CO),...,SWSW,NaN,NaN,NaN,NaN,39.712224,-104.543270,39.712224,-104.543270,"=""05-005-06864-00"""
1,"=""0500506866""","=""050050686600""","=""05005068660000""",STATE-CRAIG,2-16,STATE-CRAIG,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SESW,NaN,NaN,NaN,NaN,39.609786,-104.334593,39.609786,-104.334593,"=""05-005-06866-00"""
2,"=""0500506867""","=""050050686700""","=""05005068670000""",LINNEBUR,1-20,LINNEBUR,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SESE,NaN,NaN,NaN,NaN,39.595196,-104.344133,39.595196,-104.344133,"=""05-005-06867-00"""
3,"=""0500506870""","=""050050687000""","=""05005068700000""",COLUMBINE-STATE,2,COLUMBINE-STATE,AXIS ENERGY,AXIS EXPLORATION LLC,CHALICE,ARAPAHOE (CO),...,SENE,NaN,NaN,NaN,NaN,39.706044,-104.547569,39.706044,-104.547569,"=""05-005-06870-00"""
4,"=""0500506876""","=""050050687600""","=""05005068760000""",STATE-CRAIG,3-16,STATE-CRAIG,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SWSE,NaN,NaN,NaN,NaN,39.609906,-104.330093,39.609906,-104.330093,"=""05-005-06876-00"""


Alright, this is some fairly clean data, but I already see a couple of spots that need cleaning, such as API numbers, and some columns that might need to be dropped because of their lack of data.  Let's now check out all of the columns in this data set.

In [6]:
wells.columns

Index(['API10', 'API12', 'API14', 'Well Name', 'Well Number', 'Lease Name',
       'Operator Alias', 'Reported Operator', 'Field', 'County/Parish',
       'Target Formation', 'DI Landing Zone', 'KB Elevation',
       'Ground Elevation', 'Measured Depth (TD)', 'True Vertical Depth',
       'Production Type', 'Well Status', 'Spud Date', 'Completion Date',
       'Drill Type', 'Upper Perforation', 'Lower Perforation',
       'Gross Perforated Interval', 'Horizontal Length',
       'Wellbore Count (API10)', 'Completion Count (API10)',
       'Treatment Job Count (API10)', 'First Prod Date',
       'Start Date (First Treatment Job)', 'End Date (First Treatment Job)',
       'First Test Gas Volume', 'First Test Oil Volume',
       'First Test Water Volume', 'First Test Hours Tested',
       'First Test Production Method', 'First Well Test Date',
       'First Reported Test Type', 'Last Well Test Date', 'Last Test Type',
       'First Test Gas Gravity', 'First Test Oil Gravity', 'First Test G

Columns I'm interested in and an explanation of what they contain:
- API14 - Well ID
- Well Name - Name of the well
- Well Number - Number of well name
- Operator Alias - Company Name
- Field - General location
- Target Formation - Type of Rock that is being drilled/fraced
- Measured Depth (TD) - Total depth of well (vertical + horizontal distance)
- True Vertical Depth - Vertical depth of well
- Drill Type - Type of well (vertical, horizontal, or distributed)
- Upper Perforation - Depth of first perforation (mostly for vertical wells)
- Lower Perforation - Depth of last perforation (mostly for horizontal wells)
- Gross Perforated Interval - How frequent the cement was perforated 
- Horizontal Length - Length of the horizontal portion of the well
- Treatment Job Count (API10) - The number of times the well was fraced
- Cum BOE - Cumulative Barrels of Oil Equivalent (from oil and gas produced) 
- Cum Gas - Cumulative gas produced
- Cum Oil - Cumulative oil produced
- First 6 BOE - Barrels of oil equvalent produced in first 6 months of production
- Township - Location North and South of base line
- Range - Location East and West of meridian

In [7]:
# Create new wells data frame with only columns of interest and horizontal drill types.
wells1 = wells[['API14', 
                'Well Name',
                'Well Number',
                'Operator Alias', 
                'Field',
                'Target Formation', 
                'Measured Depth (TD)',
                'True Vertical Depth', 
                'Drill Type',
                'Upper Perforation', 
                'Lower Perforation',
                'Gross Perforated Interval', 
                'Horizontal Length', 
                'Treatment Job Count (API10)',
                'Cum BOE', 
                'Cum Gas', 
                'Cum Oil', 
                'First 6 BOE', 
                'First Prod Date', 
                'Township', 
                'Range',
                     ]]
hzwells = wells1[wells1['Drill Type'] == 'H']
hzwells.head()

,API14,Well Name,Well Number,Operator Alias,Field,Target Formation,Measured Depth (TD),True Vertical Depth,Drill Type,Upper Perforation,...,Gross Perforated Interval,Horizontal Length,Treatment Job Count (API10),Cum BOE,Cum Gas,Cum Oil,First 6 BOE,First Prod Date,Township,Range
6002,"=""05123192590000""",BRINGELSON,1-28X,PETERSON ENERGY OPERATING,NEW DAY,NIOBRARA,7630.0,5827.0,H,6185.0,...,1230.0,NaN,1.0,474.0,0.0,474.0,448.0,1997-03-01,"=""09N""",58W
10040,"=""05005072210100""",STATE MASSIVE,1H,CONOCO PHILLIPS,DJ HORIZONTAL NIOBRARA,NIOBRARA,17291.0,8008.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"=""05S""",65W
10059,"=""05005073270100""",STATE CHALLENGER 5-65 2-3,3BYH,CONOCO PHILLIPS,WILDCAT,NIOBRARA,17847.0,7983.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"=""05S""",65W
15865,"=""05123343680100""",BARRACUDA,29-11-7-60,VERDAD RESOURCES LLC,WILDCAT,NIOBRARA,10697.0,6351.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"=""07N""",60W
16075,"=""05123351000100""",RAINDANCE FC,23-369HC,GREAT WESTERN OPERATING,WATTENBERG,CODELL,20186.0,6580.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"=""06N""",67W


Great - now only having the columns of interest, I can zoom in on the data that is relevant to this project to see what needs to be cleaned, what is missing, etc.

In [8]:
hzwells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15092 entries, 6002 to 60908
Data columns (total 21 columns):
API14                          15092 non-null object
Well Name                      15092 non-null object
Well Number                    15092 non-null object
Operator Alias                 14312 non-null object
Field                          15092 non-null object
Target Formation               15092 non-null object
Measured Depth (TD)            14242 non-null float64
True Vertical Depth            12006 non-null float64
Drill Type                     15092 non-null object
Upper Perforation              4987 non-null float64
Lower Perforation              4986 non-null float64
Gross Perforated Interval      4983 non-null float64
Horizontal Length              15068 non-null float64
Treatment Job Count (API10)    6624 non-null float64
Cum BOE                        6265 non-null float64
Cum Gas                        6233 non-null float64
Cum Oil                        6259 n

Alright, it looks like there are about 6K entries that include the first 6 months of BOE, which will be my target output at least to start.  The other wells must be pre-production, didn't track this value, or did not produce.  Let's pull out these wells.

In [9]:
hzwells_pr = hzwells[hzwells['First 6 BOE'] >0]
hzwells_pr.head()

,API14,Well Name,Well Number,Operator Alias,Field,Target Formation,Measured Depth (TD),True Vertical Depth,Drill Type,Upper Perforation,...,Gross Perforated Interval,Horizontal Length,Treatment Job Count (API10),Cum BOE,Cum Gas,Cum Oil,First 6 BOE,First Prod Date,Township,Range
6002,"=""05123192590000""",BRINGELSON,1-28X,PETERSON ENERGY OPERATING,NEW DAY,NIOBRARA,7630.0,5827.0,H,6185.0,...,1230.0,NaN,1.0,474.0,0.0,474.0,448.0,1997-03-01,"=""09N""",58W
17938,"=""05001097530000""",WEP,4-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11431.0,7551.0,H,7519.0,...,3912.0,4376.37,2.0,28664.0,45552.0,21072.0,10662.0,2017-04-01,"=""03S""",64W
17940,"=""05001097540000""",WEP,1-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11715.0,7594.0,H,8151.0,...,3564.0,3575.27,2.0,33546.0,60174.0,23517.0,13185.0,2013-02-01,"=""03S""",64W
17955,"=""05001098010000""",SHARP,24-3-11HC,WARD PETR,WATTENBERG,CODELL,10161.0,7774.0,H,NaN,...,NaN,2247.84,1.0,188242.0,273498.0,142659.0,46241.0,2015-01-01,"=""01S""",67W
17977,"=""05001098260000""",ALICIA,12-15H-5N,WARD PETR,SPINDLE,NIOBRARA,13842.0,7821.0,H,NaN,...,NaN,5524.53,1.0,159344.0,348837.0,101205.0,53409.0,2015-09-01,"=""01S""",68W


In [10]:
# Write DataFrame to csv for use in database
hzwells_pr.to_csv('1a_Frac_Capstone/HZwells.csv')

### Production Time Series Data
Next, let's take a look at the time series data to see what's there.

In [11]:
prod_time = pd.read_csv('1a_Frac_Capstone/dj hz 6-17-18 Production Time Series.csv')
print(prod_time.shape)
prod_time.tail()

(236016, 19)


,Entity ID,API/UWI,API/UWI List,Monthly Production Date,Monthly Oil,Monthly Gas,Monthly Water,Well Count,Days,Daily Avg Oil,Daily Avg Gas,Daily Avg Water,Reservoir,Well/Lease Name,Well Number,Operator Alias,Production Type,Production Status,Entity Type
236011,104208052,5123144240000,5.123144e+10,1998-11-01,22.0,393.0,0.0,1,0.0,0.0,13.0,0.0,CODELL,HSR-KING,4-23,"HS RESOURCES, INC.",OIL,INACTIVE,WELL
236012,104208052,5123144240000,5.123144e+10,1998-12-01,22.0,387.0,0.0,1,0.0,0.0,12.0,0.0,CODELL,HSR-KING,4-23,"HS RESOURCES, INC.",OIL,INACTIVE,WELL
236013,104208052,5123144240000,5.123144e+10,1999-01-01,60.0,935.0,0.0,1,29.0,2.0,32.0,0.0,CODELL,HSR-KING,4-23,"HS RESOURCES, INC.",OIL,INACTIVE,WELL
236014,104208052,5123144240000,5.123144e+10,1999-02-01,49.0,705.0,0.0,1,28.0,1.0,25.0,0.0,CODELL,HSR-KING,4-23,"HS RESOURCES, INC.",OIL,INACTIVE,WELL
236015,104208052,5123144240000,5.123144e+10,1999-09-01,1.0,0.0,0.0,1,30.0,0.0,0.0,0.0,CODELL,HSR-KING,4-23,"HS RESOURCES, INC.",OIL,INACTIVE,WELL


Great.  I might explore this more in depth after my initial models to determine how production changes over time, not just the total of the first six months.

## FracFocus Data
The final set of data that I was provided was from a website called [FracFocus](http://fracfocusdata.org/), which contains data on the actual frac conditions for each well.  This data set was extremely large, so I processed it through SQL, including joining on the API number with the horizontal wells data and then exported the resultant csv.

### SQL Query
`SELECT 
SUBSTR(Wells.API14, 3, 14) APIclean,
Wells.*,
sub.*,
(sub.Slickwater + sub.Gel + sub.Crosslinked) totadditives
FROM (
SELECT APINumber,
CAST(StateNumber as Integer) as StateNum,
MAX(CASE WHEN Purpose LIKE '%friction%' THEN 1 
WHEN Purpose LIKE '%mobility%' THEN 1 
WHEN Purpose LIKE '%surfactant%' THEN 1 
WHEN Purpose LIKE '%flow%' THEN 1
WHEN Purpose LIKE '%breaker%' THEN 1 ELSE 0 END) as 'Slickwater',
MAX(CASE WHEN Purpose LIKE '%gel%' THEN 1 ELSE 0 END) as 'Gel',
MAX(CASE WHEN Purpose LIKE '%cross%' THEN 1
WHEN Purpose LIKE '%x-link%' THEN 1 
WHEN Purpose LIKE '%initiator%' THEN 1 
WHEN Purpose LIKE '%guar%' THEN 1 
WHEN Purpose LIKE '%catalyst%' THEN 1 ELSE 0 END) as 'Crosslinked'
FROM FracFocusRegistry Frac
WHERE StateNum = 5 
GROUP BY APINumber
) sub 
JOIN HZWells Wells
ON APIclean = sub.APINumber
WHERE totadditives >0`

In [12]:
# Read in new wells data with frac conditions
frac_wells = pd.read_csv('1a_Frac_Capstone/FracTypeWells.csv')
frac_wells.head()

,APIclean,field1,API14,WellName,WellNumber,OperatorAlias,Field,TargetFormation,MeasuredDepth(TD),TrueVerticalDepth,...,First6BOE,FirstProdDate,Township,Range,APINumber,StateNum,Slickwater,Gel,Crosslinked,totadditives
0,5001098010000,17955,"=""05001098010000""",SHARP,24-3-11HC,WARD PETR,WATTENBERG,CODELL,10161.00,7774.00,...,46241.0,2015-01-01,"=""01S""",67W,5001098010000,5,1,1,1,3
1,5001097850000,17984,"=""05001097850000""",STATE OF CO,1S-66-36-1609CH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12177.31,7576.73,...,7094.0,2014-06-01,"=""01S""",66W,5001097850000,5,1,1,1,3
2,5001097830000,17986,"=""05001097830000""",STATE OF CO,1S-66-36-0108BH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12164.98,7511.68,...,8304.0,2014-06-01,"=""01S""",66W,5001097830000,5,1,1,1,3
3,5001097820000,17987,"=""05001097820000""",STATE OF CO,1S-66-36-0108CH,EXTRACTION O&G,WATTENBERG,NIOBRARA,12194.70,7574.72,...,7118.0,2014-06-01,"=""01S""",66W,5001097820000,5,1,1,1,3
4,5001097810000,17988,"=""05001097810000""",STATE OF CO,1S-66-36-1609BH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12117.48,7513.70,...,10385.0,2014-06-01,"=""01S""",66W,5001097810000,5,1,1,1,3


Awesome, let's check out what's in this dataset now and what we have to clean and what missing data we have.

In [13]:
frac_wells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4520 entries, 0 to 4519
Data columns (total 29 columns):
APIclean                    4520 non-null int64
field1                      4520 non-null int64
API14                       4520 non-null object
WellName                    4520 non-null object
WellNumber                  4520 non-null object
OperatorAlias               4520 non-null object
Field                       4520 non-null object
TargetFormation             4520 non-null object
MeasuredDepth(TD)           4512 non-null float64
TrueVerticalDepth           4510 non-null float64
DrillType                   4520 non-null object
UpperPerforation            3710 non-null float64
LowerPerforation            3710 non-null float64
GrossPerforatedInterval     3710 non-null float64
HorizontalLength            4519 non-null float64
TreatmentJobCount(API10)    4519 non-null float64
CumBOE                      4520 non-null float64
CumGas                      4520 non-null float64
CumO

Aside from the perforation data and a few measured depths, we have all of the other relevant data ready for some cleaning and further analysis!